In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import itertools
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

tf.random.set_seed(42)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')


df = pd.read_csv('https://raw.githubusercontent.com/Neha-Chiluka/deeplearning/refs/heads/main/tensorflow/data/winequalityN.csv')
df.sample(5)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1681,white,6.3,0.31,0.30,10.0,0.046,49.0,212.0,0.99620,3.74,0.55,11.9,6
2303,white,6.9,0.44,0.18,11.8,0.051,26.0,126.0,0.99750,3.23,0.48,9.1,5
2903,white,5.7,0.21,0.25,1.1,0.035,26.0,81.0,0.99020,3.31,0.52,11.4,6
223,white,6.5,0.19,0.30,0.8,0.043,33.0,144.0,0.99360,3.42,0.39,9.1,6
2548,white,6.3,0.26,0.42,7.1,0.045,62.0,209.0,0.99544,3.20,0.53,9.5,6


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Prepare the data
df = df.dropna()
df['is_white_wine'] = [1 if typ == 'white' else 0 for typ in df['type']]
df['is_good_wine'] = [1 if quality >= 6 else 0 for quality in df['quality']]
df.drop(['type', 'quality'], axis=1, inplace=True)

# Train/test split
X = df.drop('is_good_wine', axis=1)
y = df['is_good_wine']
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:

num_layers = 3
min_nodes_per_layer = 64
max_nodes_per_layer = 256
node_step_size = 64

In [ ]:
node_options = list(range(
    min_nodes_per_layer,
    max_nodes_per_layer + 1,
    node_step_size
))
print(node_options)

[64, 128, 192, 256]


In [ ]:
two_layer_possibilities = [node_options, node_options]
print(two_layer_possibilities)

[[64, 128, 192, 256], [64, 128, 192, 256]]


In [ ]:
list(itertools.product(*two_layer_possibilities))

[(64, 64),
 (64, 128),
 (64, 192),
 (64, 256),
 (128, 64),
 (128, 128),
 (128, 192),
 (128, 256),
 (192, 64),
 (192, 128),
 (192, 192),
 (192, 256),
 (256, 64),
 (256, 128),
 (256, 192),
 (256, 256)]

In [ ]:
layer_possibilities = [node_options] * num_layers
layer_node_permutations = list(itertools.product(*layer_possibilities))
print(layer_node_permutations)

[(64, 64, 64), (64, 64, 128), (64, 64, 192), (64, 64, 256), (64, 128, 64), (64, 128, 128), (64, 128, 192), (64, 128, 256), (64, 192, 64), (64, 192, 128), (64, 192, 192), (64, 192, 256), (64, 256, 64), (64, 256, 128), (64, 256, 192), (64, 256, 256), (128, 64, 64), (128, 64, 128), (128, 64, 192), (128, 64, 256), (128, 128, 64), (128, 128, 128), (128, 128, 192), (128, 128, 256), (128, 192, 64), (128, 192, 128), (128, 192, 192), (128, 192, 256), (128, 256, 64), (128, 256, 128), (128, 256, 192), (128, 256, 256), (192, 64, 64), (192, 64, 128), (192, 64, 192), (192, 64, 256), (192, 128, 64), (192, 128, 128), (192, 128, 192), (192, 128, 256), (192, 192, 64), (192, 192, 128), (192, 192, 192), (192, 192, 256), (192, 256, 64), (192, 256, 128), (192, 256, 192), (192, 256, 256), (256, 64, 64), (256, 64, 128), (256, 64, 192), (256, 64, 256), (256, 128, 64), (256, 128, 128), (256, 128, 192), (256, 128, 256), (256, 192, 64), (256, 192, 128), (256, 192, 192), (256, 192, 256), (256, 256, 64), (256, 256,

In [ ]:
for permutation in layer_node_permutations[:2]:
    for nodes_at_layer in permutation:
        print(nodes_at_layer)
    print()

64
64
64

64
64
128



In [ ]:
models = []

for permutation in layer_node_permutations:
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(12,)))
    model_name = ''

    for nodes_at_layer in permutation:
        model.add(tf.keras.layers.Dense(nodes_at_layer, activation='relu'))
        model_name += f'dense{nodes_at_layer}_'

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model._name = model_name[:-1]

    models.append(model)

In [ ]:
models[0].summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,217 (36.00 KB)

 Trainable params: 9,217 (36.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def get_models(num_layers: int,
               min_nodes_per_layer: int,
               max_nodes_per_layer: int,
               node_step_size: int,
               input_shape: tuple,
               hidden_layer_activation: str = 'relu',
               num_nodes_at_output: int = 1,
               output_layer_activation: str = 'sigmoid') -> list:

    node_options = list(range(min_nodes_per_layer, max_nodes_per_layer + 1, node_step_size))
    layer_possibilities = [node_options] * num_layers
    layer_node_permutations = list(itertools.product(*layer_possibilities))

    models = []
    for permutation in layer_node_permutations:
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
        model_name = ''

        for nodes_at_layer in permutation:
            model.add(tf.keras.layers.Dense(nodes_at_layer, activation=hidden_layer_activation))
            model_name += f'dense{nodes_at_layer}_'

        model.add(tf.keras.layers.Dense(num_nodes_at_output, activation=output_layer_activation))
        model._name = model_name[:-1]
        models.append(model)

    return models

In [ ]:
all_models = get_models(
    num_layers=3,
    min_nodes_per_layer=64,
    max_nodes_per_layer=256,
    node_step_size=64,
    input_shape=(12,)
)
print(all_models)

[<Sequential name=sequential_128, built=True>, <Sequential name=sequential_129, built=True>, <Sequential name=sequential_130, built=True>, <Sequential name=sequential_131, built=True>, <Sequential name=sequential_132, built=True>, <Sequential name=sequential_133, built=True>, <Sequential name=sequential_134, built=True>, <Sequential name=sequential_135, built=True>, <Sequential name=sequential_136, built=True>, <Sequential name=sequential_137, built=True>, <Sequential name=sequential_138, built=True>, <Sequential name=sequential_139, built=True>, <Sequential name=sequential_140, built=True>, <Sequential name=sequential_141, built=True>, <Sequential name=sequential_142, built=True>, <Sequential name=sequential_143, built=True>, <Sequential name=sequential_144, built=True>, <Sequential name=sequential_145, built=True>, <Sequential name=sequential_146, built=True>, <Sequential name=sequential_147, built=True>, <Sequential name=sequential_148, built=True>, <Sequential name=sequential_149, 

In [ ]:
def optimize(models: list,
             X_train: np.array,
             y_train: np.array,
             X_test: np.array,
             y_test: np.array,
             epochs: int = 50,
             verbose: int = 0) -> pd.DataFrame:

    # We'll store the results here
    results = []

    def train(model: tf.keras.Sequential) -> dict:
        # Change this however you want
        # We're not optimizing this part today
        model.compile(
            loss=tf.keras.losses.binary_crossentropy,
            optimizer=tf.keras.optimizers.Adam(),
            metrics=[
                tf.keras.metrics.BinaryAccuracy(name='accuracy')
            ]
        )

        # Train the model
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            verbose=verbose
        )

        # Make predictions on the test set
        preds = model.predict(X_test)
        prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(preds)]

        # Return evaluation metrics on the test set
        return {
            'model_name': model.name,
            'test_accuracy': accuracy_score(y_test, prediction_classes),
            'test_precision': precision_score(y_test, prediction_classes),
            'test_recall': recall_score(y_test, prediction_classes),
            'test_f1': f1_score(y_test, prediction_classes)
        }

    # Train every model and save results
    for model in models:
        try:
            print(model.name, end=' ... ')
            res = train(model=model)
            results.append(res)
        except Exception as e:
            print(f'{model.name} --> {str(e)}')

    return pd.DataFrame(results)

In [ ]:
optimization_results = optimize(
    models=all_models,
    X_train=X_train_scaled,
    y_train=y_train,
    X_test=X_test_scaled,
    y_test=y_test
)

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━

In [ ]:
optimization_results.sort_values(by='test_accuracy', ascending=False)

,model_name,test_accuracy,test_precision,test_recall,test_f1
59,sequential_187,0.813612,0.846715,0.858200,0.852419
43,sequential_171,0.810518,0.836905,0.866831,0.851605
53,sequential_181,0.808971,0.832547,0.870530,0.851115
34,sequential_162,0.808198,0.823936,0.882861,0.852381
40,sequential_168,0.807425,0.837740,0.859433,0.848448
...,...,...,...,...,...
62,sequential_190,0.780356,0.811098,0.847102,0.828709
18,sequential_146,0.776489,0.769628,0.918619,0.837549
50,sequential_178,0.775715,0.809015,0.840937,0.824667
0,sequential_128,0.774169,0.806375,0.842170,0.823884
